In [1]:
# Check GPU memory, clear and recheck
from numba import cuda
gpu = cuda.select_device(0)
print('Selected GPU is:', gpu)
memory = cuda.current_context().get_memory_info()
print('Memory Status:', memory[0], 'free out of', memory[1], ',', (int(memory[0])/int(memory[1])*100), '% free')

# print('Clearing Memory...')
# gpu = cuda.select_device(0)
# gpu.reset()
# memory = cuda.current_context().get_memory_info()
# print('Memory Status:', memory[0], 'free out of', memory[1], ',', (int(memory[0])/int(memory[1])*100), '% free')

Selected GPU is: <CUDA device 0 'b'Quadro M4000''>
Memory Status: 7200217497 free out of 8589934592 , 83.8215637137182 % free


In [1]:
"""
Created on Wed Feb 26 17:01:56 2020

"""

""" IMPORTS """
import sys
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.constraints import *
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

from binary_ops import binary_tanh as binary_tanh_op
from binary_layers import BinaryDense, BinaryConv2D

import h5py
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
from lambda_layers import *
%matplotlib qt

In [27]:
""" FUNCTIION AND VARIABLE DEFINITIONS """
def binary_tanh(x):
    return binary_tanh_op(x)

H = 1.
kernel_lr_multiplier = 'Glorot'

# # nn
batch_size = 50
epochs = 20
channels = 1
img_rows = 30
img_cols = 30
filters = 32
kernel_size = (32, 32)
pool_size = (2, 2)
hidden_units = 128
classes = 10
use_bias = False

# # learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# # BN
epsilon = 1e-6
momentum = 0.9

# # dropout
p1 = 0.25
p2 = 0.5

hdf5_dir = Path("../data/hdf5/")

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read
        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images= []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_vids.h5", "r+")

    images = np.array(file["/images"]).astype("float32")

    return images

def np_streak(x):
    input_dims = np.shape(x)
    output_shape = (input_dims[0],input_dims[1],input_dims[1]+input_dims[2],input_dims[3],input_dims[4])
    streak_tensor = np.zeros(output_shape)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            streak_tensor[i,j,j:(output_shape[3]+j),:,:] = x[i,j,:,:,:]
    #return streak_tensor
    return np.sum(streak_tensor,axis=1)

def mask(val,ims,mask):
    for i in range(np.shape(val)[0]):
        for j in range(np.shape(val)[1]):
            val[i,j,:,:] = ims[i,j,:,:] * mask
    return val

ACCURACY_THRESHOLD = 0.035

class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('val_loss') < ACCURACY_THRESHOLD):
			print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
			self.model.stop_training = True

# Instantiate a callback object
threshold = myCallback()

def custom_loss(y_true, y_pred):

  ssim_loss = (1.0-tf.image.ssim(y_true,y_pred,1))/2.0
  mse_loss = K.mean(K.square(y_pred-y_true))
  #mse_loss = tf.keras.losses.mean_squared_error(y_true,y_pred)

  ssim_loss = 0.3*ssim_loss
  mse_loss = 0.7*mse_loss

  return ssim_loss + mse_loss

def ssim_loss(y_true,y_pred):  
    return (1.0-tf.image.ssim(y_true,y_pred,1))/2.0

def mse_loss(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true))


### 3943
# ims = read_many_hdf5(3943)
# # ims = np.ones((3943,30,32,32,1))
# ims = np.reshape(ims, (-1,30,32,32,1))
# ims = ims[:3900]
# # temp = np.zeros((1,32,32,1))
# validate2 = np.zeros((3900,30,32,32,1))
# bk_temp = np.random.randint(0,2,(1,32,32,1))
# validate2 = mask(validate2,ims,bk_temp)
# ims2 = np_streak(validate2)

#### 1515
#ims = read_many_hdf5(1516)
# ims = np.ones((3943,30,32,32,1))
#ims = np.reshape(ims, (-1,30,32,32,1))
#ims = ims[:1500]
# temp = np.zeros((1,32,32,1))
#validate2 = np.zeros((1500,30,32,32,1))
#bk_temp = np.random.randint(0,2,(1,32,32,1))
#validate2 = mask(validate2,ims,bk_temp)
#ims2 = np_streak(validate2)

ims = read_many_hdf5(784)
# ims = np.ones((3943,30,32,32,1))
ims = np.reshape(ims, (-1,30,32,32,1))
ims = ims[:750]
# temp = np.zeros((1,32,32,1))
validate2 = np.zeros((750,30,32,32,1))
bk_temp = np.random.randint(0,2,(1,32,32,1))
validate2 = mask(validate2,ims,bk_temp)
ims2 = np_streak(validate2)


validate = ims

validate = validate / 255
ims2 = ims2 /255
ims = ims/255
#X_train, X_test, y_train, y_test = train_test_split(ims, validate, test_size=(1/3), random_state=42)
X_train, X_test, y_train, y_test = train_test_split(ims2, validate, test_size=(1/3), random_state=42)

MX_train, MX_test, My_train, My_test = train_test_split(ims,ims, test_size = 1/3, random_state = 42)

print(np.shape(X_test))
print(np.shape(X_train))

reduce_lr = ReduceLROnPlateau(monitor='val_loss',verbose=1, factor=0.2,
                              patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(patience=20,verbose=1,restore_best_weights=True)   





(250, 62, 32, 1)
(500, 62, 32, 1)


In [4]:
fix, ax = plt.subplots(nrows = 1, ncols = 4)
i2t = np.reshape(MX_train,(-1,30,32,32))
for col in range(4):
    ax[col].imshow(i2t[10+(6*col)][10],cmap="gray")

In [1]:
""" MODEL DEFINITION """
model = Sequential()
model.add(Input(shape=(30,32,32,1),batch_size = 50))
model.add(TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1')))
model.add(Lambda(streak,output_shape=streak_output_shape))
model.add(Lambda(integrate_ims, output_shape = integrate_ims_output_shape))
model.add(Flatten())
model.add(Dense(30720, activation = 'relu'))
model.add(Reshape((30,32,32,1)))
model.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = ['mean_squared_error'])
model.layers[0].trainable=False
model.summary()
history = model.fit(MX_train, My_train,
          batch_size = 50,epochs= 100,
          verbose=2,validation_data=(MX_test,My_test))

NameError: name 'Sequential' is not defined

In [29]:

y_pred_ssim = model.predict(MX_train)

In [30]:
y_pred_ssim = tf.convert_to_tensor(y_pred_ssim)
y_true_ssim = tf.convert_to_tensor(My_train)

print("AVG SSIM: " + str(ssim_loss(y_pred_ssim, y_true_ssim).numpy().mean()))
print("AVG MSE: " + str(mse_loss(y_pred_ssim,y_true_ssim).numpy().mean()))
## sanity check

o = tf.convert_to_tensor(np.ones((30,30,1)))
z = tf.convert_to_tensor(np.zeros((30,30,1)))
ssim_loss(o,z)
tf.image.ssim(o,z,1)

AVG SSIM: 0.44480523
AVG MSE: 0.09995544


<tf.Tensor: id=58306, shape=(), dtype=float32, numpy=0.0001000032>

In [ ]:
m1_y_pred = model.predict(MX_test)
show_video(m1_y_pred,MX_test,15)

In [39]:
from binary_ops import *
get_mask(model)

NameError: name 'get_mask' is not defined

In [40]:
def get_mask(model,l=0):
    b = binarize(model.layers[l].weights[0])
    figb,axb = plt.subplots(1,1)
    axb.imshow(np.reshape(b,(32,32)),cmap="gray")

In [31]:
inputs = Input(shape=(30,32,32,1),batch_size=50)
bin_conv1 = TimeDistributed(BinaryConv2D(1, kernel_size=(32,32), input_shape=(30,32,32,1),
                       data_format='channels_last',
                       H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                       padding='same', use_bias=use_bias, name='bin_conv_1'))(inputs)
s = Lambda(streak, output_shape = streak_output_shape)(bin_conv1)
i = Lambda(integrate_ims, output_shape = integrate_ims_output_shape) (s)
f = Flatten()(i)
dense1 = Dense(30720, activation = 'relu')(f)
resh = Reshape((30,32,32,1))(dense1)
resh = Reshape((30,32,32,1))(dense1)
c1 = TimeDistributed(Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (resh)
c1 = Dropout(0.1) (c1)
c1 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c1)
p1 = TimeDistributed(MaxPooling2D((2, 2)))(c1)

c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))(p1)
c2 = Dropout(0.1) (c2)
c2 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c2)
p2 = TimeDistributed(MaxPooling2D((2, 2)) )(c2)

c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p2)
c3 = Dropout(0.2) (c3)
c3 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c3)
p3 = TimeDistributed(MaxPooling2D((2, 2)) )(c3)
    
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(p3)
c4 = Dropout(0.2) (c4)
c4 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(c4)
p4 = TimeDistributed(MaxPooling2D(pool_size=(2, 2))) (c4)

c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (p4)
c5 = Dropout(0.3) (c5)
c5 = TimeDistributed(Conv2D(256, (2, 2), activation='relu', kernel_initializer='he_normal', padding='same')) (c5)

u6 = TimeDistributed(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))(c5)
u6 = concatenate([u6, c4])
c6 = TimeDistributed( Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') )(u6)
c6 = Dropout(0.2) (c6)
c6 = TimeDistributed(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c6)

u7 = TimeDistributed(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same') )(c6)
u7 = concatenate([u7, c3])
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u7)
c7 = Dropout(0.2) (c7)
c7 = TimeDistributed(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c7)
    
u8 = TimeDistributed(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same') )(c7)
u8 = concatenate([u8, c2])
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u8)
c8 = Dropout(0.1) (c8)
c8 = TimeDistributed(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c8)

u9 = TimeDistributed(Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')) (c8)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (u9)
c9 = Dropout(0.1) (c9)
c9 = TimeDistributed(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')) (c9)
    
outputs = TimeDistributed(Conv2D(1, (1, 1), activation='sigmoid')) (c9)

CUPNET = Model(inputs = [inputs], outputs = [outputs])
    
CUPNET.compile(optimizer = Nadam(0.0001), loss = custom_loss, metrics = ['mean_squared_error'])

In [32]:
CUPNET.summary()
cupnet_history = CUPNET.fit(MX_train, My_train,
          batch_size = 50,epochs= 500,
          verbose=2,validation_data=(MX_test,My_test))



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(50, 30, 32, 32, 1) 0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (50, 30, 32, 32, 1)  1024        input_9[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (50, 30, 62, 32, 1)  0           time_distributed_2[0][0]         
__________________________________________________________________________________________________
lambda_5 (Lambda)               (50, 62, 32, 1)      0           lambda_4[0][0]                   
______________________________________________________________________________________________

500/500 - 21s - loss: 0.1785 - mean_squared_error: 0.0712 - val_loss: 0.1783 - val_mean_squared_error: 0.0747
Epoch 2/500
500/500 - 8s - loss: 0.1734 - mean_squared_error: 0.0696 - val_loss: 0.1770 - val_mean_squared_error: 0.0737
Epoch 3/500
500/500 - 8s - loss: 0.1718 - mean_squared_error: 0.0684 - val_loss: 0.1757 - val_mean_squared_error: 0.0721
Epoch 4/500
500/500 - 8s - loss: 0.1702 - mean_squared_error: 0.0663 - val_loss: 0.1734 - val_mean_squared_error: 0.0695
Epoch 5/500
500/500 - 8s - loss: 0.1680 - mean_squared_error: 0.0634 - val_loss: 0.1703 - val_mean_squared_error: 0.0661
Epoch 6/500
500/500 - 8s - loss: 0.1661 - mean_squared_error: 0.0613 - val_loss: 0.1676 - val_mean_squared_error: 0.0634
Epoch 7/500
500/500 - 8s - loss: 0.1642 - mean_squared_error: 0.0593 - val_loss: 0.1660 - val_mean_squared_error: 0.0622
Epoch 8/500
500/500 - 8s - loss: 0.1624 - mean_squared_error: 0.0575 - val_loss: 0.1649 - val_mean_squared_error: 0.0612
Epoch 9/500
500/500 - 8s - loss: 0.1610 - m

Epoch 69/500
500/500 - 8s - loss: 0.1252 - mean_squared_error: 0.0269 - val_loss: 0.1304 - val_mean_squared_error: 0.0308
Epoch 70/500
500/500 - 8s - loss: 0.1244 - mean_squared_error: 0.0264 - val_loss: 0.1293 - val_mean_squared_error: 0.0294
Epoch 71/500
500/500 - 8s - loss: 0.1250 - mean_squared_error: 0.0270 - val_loss: 0.1366 - val_mean_squared_error: 0.0367
Epoch 72/500
500/500 - 8s - loss: 0.1234 - mean_squared_error: 0.0256 - val_loss: 0.1383 - val_mean_squared_error: 0.0391
Epoch 73/500
500/500 - 8s - loss: 0.1240 - mean_squared_error: 0.0262 - val_loss: 0.1373 - val_mean_squared_error: 0.0377
Epoch 74/500
500/500 - 8s - loss: 0.1231 - mean_squared_error: 0.0256 - val_loss: 0.1404 - val_mean_squared_error: 0.0416
Epoch 75/500
500/500 - 8s - loss: 0.1253 - mean_squared_error: 0.0275 - val_loss: 0.1357 - val_mean_squared_error: 0.0359
Epoch 76/500
500/500 - 8s - loss: 0.1232 - mean_squared_error: 0.0257 - val_loss: 0.1313 - val_mean_squared_error: 0.0315
Epoch 77/500
500/500 - 8

KeyboardInterrupt: 

In [33]:
y_pred_ssim = CUPNET.predict(MX_train)
y_pred_ssim = tf.convert_to_tensor(y_pred_ssim)
y_true_ssim = tf.convert_to_tensor(My_train)

print("AVG SSIM: " + str(ssim_loss(y_pred_ssim, y_true_ssim).numpy().mean()))
print("AVG MSE: " + str(mse_loss(y_pred_ssim,y_true_ssim).numpy().mean()))
## sanity check

o = tf.convert_to_tensor(np.ones((30,30,1)))
z = tf.convert_to_tensor(np.zeros((30,30,1)))
ssim_loss(o,z)
tf.image.ssim(o,z,1)

AVG SSIM: 0.35070547
AVG MSE: 0.035765294


<tf.Tensor: id=73643, shape=(), dtype=float32, numpy=0.0001000032>

In [34]:
def show_video(y_pred,y_true,num):
    yp = np.reshape(y_pred,(-1,30,32,32))
    yt = np.reshape(y_true,(-1,30,32,32))
    fig,ax = plt.subplots(nrows=5,ncols=6,sharex=True,sharey=True)
    fig2,ax2 = plt.subplots(nrows=5,ncols=6,sharex=True,sharey=True)
    for row in range(5):
        for col in range(6):
            ax[row,col].imshow(yp[num][5*row+col],cmap="gray")
            ax2[row,col].imshow(yt[num][5*row+col],cmap="gray")

In [36]:
y_pred = CUPNET.predict(MX_test)
y_pred = np.reshape(y_pred,(-1,30,32,32))
fix,ax = plt.subplots(nrows=1,ncols=2)
ax[0].imshow(y_pred[110][10],cmap="gray")
ax[1].imshow(np.reshape(MX_test[110][10],(32,32)),cmap="gray")

In [41]:
from binary_ops import *
get_mask(CUPNET,1)
show_video(y_pred,MX_test,15)

In [35]:
show_video(y_pred,MX_test,111)

NameError: name 'y_pred' is not defined

In [ ]:
def show_all_videos(videos,rows,cols):
    yp = np.reshape(videos, (-1,30,32,32))
    fix3,ax3 = plt.subplots(nrows=rows, ncols = cols)
    for row in range(rows):
        for col in range(cols):
            ax3[row,col].imshow(yp[rows*row+col][3],cmap="gray")

In [ ]:
show_all_videos(y_pred,10,25)

In [ ]:
ims = np.reshape(ims, (-1,30,32,32,1))
ims = ims/255
y_pred = model2.predict(ims2[0:100])
print(np.shape(y_pred))
y_pred = np.reshape(y_pred,(-1,30,32,32))
plt.imshow(y_pred[10][10],cmap="gray")
y_pred[10][10].mean()

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=6, sharex=True, sharey=True,)
count = 0
for idx in range(5):    
  for jdx in range(6): 
    ax[idx,jdx].imshow(y_pred[10][count],cmap="gray")
    count+=1

In [ ]:
ims = np.reshape(ims,(-1,30,32,32))
fig, ax = plt.subplots(nrows=5, ncols=6, sharex=True, sharey=True,)
count = 0
for idx in range(5):    
  for jdx in range(6): 
    ax[idx,jdx].imshow(ims[10][idx+jdx],cmap="gray")

In [ ]:
tmap = ims[10][0] == ims[10][1]
tmap2 = y_pred[10][0] == y_pred[10][29]
fix2,ax2 = plt.subplots(nrows=4,ncols=3,sharex=True,sharey=True)
ax2[0,0].imshow(ims[10][0],cmap="gray")
ax2[0,1].imshow(ims[10][29],cmap="gray")
ax2[0,2].imshow(tmap,cmap="gray")
ax2[1,0].imshow(y_pred[10][0],cmap="gray")
ax2[1,1].imshow(y_pred[10][29],cmap="gray")
ax2[1,2].imshow(tmap2,cmap="gray")
ax2[2,0].imshow(ims[88][0],cmap="gray")
ax2[2,1].imshow(ims[88][29],cmap="gray")
ax2[2,2].imshow(tmap,cmap="gray")
ax2[3,0].imshow(y_pred[88][0],cmap="gray")
ax2[3,1].imshow(y_pred[88][29],cmap="gray")
ax2[3,2].imshow(tmap2,cmap="gray")

In [ ]:
fig3, ax3 = plt.subplots(nrows= 4, ncols=4,sharex=True,sharey=True)
for idx in range(4):
    for jdx in range(4):
        ax3[idx,jdx].imshow(y_pred[10+idx+jdx][10],cmap="gray")

In [ ]:
fig4, ax4 = plt.subplots(nrows=4, ncols=4, sharex=True,sharey=True)
for idx in range(4):
    for jdx in range(4):
        ax4[idx,jdx].imshow(ims[10+idx+jdx][10],cmap="gray")

In [ ]:
with ()